<a href="https://colab.research.google.com/github/aledima00/Project4_SemSeg_AML2024/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 4 - Semantic Segmentation

First let's download dataset, that is already split in "Train", "Test" and "Val" modules

In [1]:
import os

import logging
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import os
import glob
import numpy as np
from enum import Enum
from PIL import Image
import torchvision
from torchvision import transforms

from torch.utils.data import DataLoader

In [5]:
!wget -O "train.zip" "https://zenodo.org/records/5706578/files/Train.zip?download=1"
!unzip "train.zip" | tail -n 3

--2024-12-13 17:23:03--  https://zenodo.org/records/5706578/files/Train.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4021669263 (3.7G) [application/octet-stream]
Saving to: ‘train.zip’

train.zip           100%[===================>]   3.75G  12.6MB/s    in 5m 15s  

2024-12-13 17:28:19 (12.2 MB/s) - ‘train.zip’ saved [4021669263/4021669263]

  inflating: Train/Urban/masks_png/2519.png  
  inflating: Train/Urban/masks_png/2520.png  
  inflating: Train/Urban/masks_png/2521.png  


In [7]:
!wget -O "val.zip" "https://zenodo.org/records/5706578/files/Val.zip?download=1"
!unzip "val.zip" | tail -n 3

--2024-12-13 17:29:41--  https://zenodo.org/records/5706578/files/Val.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.45.92, 188.185.48.194, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2425958254 (2.3G) [application/octet-stream]
Saving to: ‘val.zip’

val.zip             100%[===================>]   2.26G  18.2MB/s    in 2m 7s   

2024-12-13 17:31:49 (18.2 MB/s) - ‘val.zip’ saved [2425958254/2425958254]

  inflating: Val/Urban/masks_png/4188.png  
  inflating: Val/Urban/masks_png/4189.png  
  inflating: Val/Urban/masks_png/4190.png  


In [8]:
!wget -O "Test.zip" "https://zenodo.org/records/5706578/files/Test.zip?download=1"
!unzip "Test.zip" | tail -n 3

--2024-12-13 17:32:09--  https://zenodo.org/records/5706578/files/Test.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.185.43.25, 188.185.48.194, 188.185.45.92, ...
Connecting to zenodo.org (zenodo.org)|188.185.43.25|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3126023212 (2.9G) [application/octet-stream]
Saving to: ‘Test.zip’

Test.zip            100%[===================>]   2.91G  18.8MB/s    in 2m 43s  

2024-12-13 17:34:53 (18.3 MB/s) - ‘Test.zip’ saved [3126023212/3126023212]

 extracting: Test/Urban/images_png/5984.png  
 extracting: Test/Urban/images_png/5985.png  
 extracting: Test/Urban/images_png/5986.png  


## Data Preparation

Create Dataset class and filter urban pictures...

In [9]:
DBG = True

DEVICE = "cpu"
TRAIN_PATH = "Train"
TEST_PATH = "Test"
VAL_PATH = "Val"

BATCH_SIZE = 1 if DBG else 128

In [10]:
# Test
# Rural_train = os.path.join(TRAIN_PATH, "Rural")
# images_rural_train = os.path.join(Rural_train, "images_png")
# os.listdir(images_rural_train)

In [11]:
# Define transforms for training phase
train_transform = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalizes tensor with mean and standard deviation
                                      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # Normalize as per ImageNet stats
])
# Define transforms for the evaluation phase
eval_transform = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                      transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))  # Normalize as per ImageNet stats
])

In [12]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')

logger = logging.getLogger(__name__)

class DataType(Enum):
    RURAL = 0
    URBAN = 1

class LoveDA(Dataset):
    def __init__(self, basedir, data_type:DataType, transforms=None, target_transform=None):
        #super(LoveDA, self).__init__(basedir, transforms, target_transform)
        if data_type == DataType.RURAL:
            self.base_path = os.path.join(basedir, "Rural")
        else: #data_type == DataType.URBAN:
            self.base_path = os.path.join(basedir, "Urban")


        # list of integers that identifies paths relative to both images_png and masks_png
        self.int_pathrefs = os.listdir(os.path.join(self.base_path, "images_png"))
        self.int_pathrefs = list(int(st.split(".")[0]) for st in self.int_pathrefs)
        self.transforms = transforms

    def __getitem__(self, idx):
      assert idx < len(self), 'Index out of range'
      image_path = os.path.join(self.base_path, "images_png", str(self.int_pathrefs[idx]) + ".png")
      mask_path = os.path.join(self.base_path, "masks_png", str(self.int_pathrefs[idx]) + ".png")
      image = pil_loader(image_path)
      mask = pil_loader(mask_path)

      if self.transforms is not None:
          image = self.transforms(image)
          mask = self.transforms(mask)
      return image, mask

    def __len__(self):
        return len(self.int_pathrefs)


In [13]:
# Urban Datasets
urban_val = LoveDA(VAL_PATH, DataType.URBAN, transforms=eval_transform)
urban_train = LoveDA(VAL_PATH, DataType.URBAN, transforms=train_transform)

# Urban Dataloader
urban_train_dataloader = DataLoader(urban_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
urban_val_dataloader = DataLoader(urban_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
# Test
# for elm in urban_train_dataloader:
#     print(elm)
#     break

Download the pretrained model with *Resnet101* as backbone

Get DeepLabv2 implementation:

In [15]:
!git clone https://github.com/Gabrysse/MLDL2024_project1.git

Cloning into 'MLDL2024_project1'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 34 (delta 8), reused 4 (delta 4), pack-reused 16 (from 1)
Receiving objects: 100% (34/34), 12.06 KiB | 398.00 KiB/s, done.
Resolving deltas: 100% (8/8), done.
